In [1]:
modello_per_analisi = 'BMW_Serie_1'
# Opzioni possibili: Mercedes_Classe_A, BMW_Serie_1, Audi_A3, Fiat_Panda

### 0. Import funzioni e librerie

In [2]:
import pandas as pd
import os
import sys
import requests
import importlib
from pathlib import Path


In [3]:
project_root = Path.cwd().parents[1]

# Percorsi delle cartelle contenenti i moduli
loader_dir = project_root / 'Source' / '2_Data_Preparation' / 'Utils'
utils_dir = project_root / 'Source' / '3_Data_Analysis' / 'Utils'
# Aggiungi al sys.path
sys.path.extend([str(loader_dir), str(utils_dir)])

# Import
from data_loader import load_csv_for_notebook
from Plots_maker import plot_is_conveniente_distribution, plot_price_prediction_distributions
from Function_price_analysis import modello_ml, predizione_prezzo, is_conveniente_class_report, indice_appetibilita, display_top_auto

In [4]:
# --- IMPORT CONFIGURAZIONE DELL'AUTO ---
#Configurazione path per import dinamico del modulo di configurazione 

config_path = project_root / 'Data'/ 'config'

sys.path.extend([str(config_path)])
print("\n - config_path:", config_path)
print(" - Contenuto della cartella:", os.listdir(config_path))

# --- Import dinamico del modulo di configurazione in base al modello ---
modulo_config = f"config_{modello_per_analisi}"  # esempio: config_Mercedes_Classe_A
try:
    config = importlib.import_module(modulo_config)
except ModuleNotFoundError:
    print(f"❌ Modulo di configurazione '{modulo_config}' non trovato.")
    sys.exit(1)
# --- Accesso ai parametri di configurazione ---
#allestimenti = config.allestimenti
allestimento_performance = config.allestimento_performance
allestimento_sport = config.allestimento_sport
allestimento_middle = config.allestimento_middle
allestimento_base = config.allestimento_base


 - config_path: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/config
 - Contenuto della cartella: ['config_Audi_A3.py', 'config_Opel_Corsa.py', '__pycache__', 'config_BMW_Serie_1.py', 'config_Mercedes_Classe_A.py']


### 1. Dati

In [5]:
#### CARICAMENTO DATI SINGOLI
data_dummy= load_csv_for_notebook(project_root,['Data', 'Processed_data', modello_per_analisi, f'data_dummy_{modello_per_analisi}.csv'])
data = load_csv_for_notebook(project_root,['Data', 'Processed_data', modello_per_analisi, f'data_{modello_per_analisi}.csv'])

X = data_dummy.drop(columns=['Prezzo', 'Distanza', 'Venditore'])
y = data_dummy['Prezzo']

✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/BMW_Serie_1/data_dummy_BMW_Serie_1.csv (2004 righe, 15 colonne)
✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/BMW_Serie_1/data_BMW_Serie_1.csv (2004 righe, 18 colonne)


### 2. Predizione prezzo, analisi della convenienza

In [6]:
modello_ml(X, y, (project_root / 'Data' / 'Modelli_ML' / modello_per_analisi))
data_ml, y_pred = predizione_prezzo(data_dummy, X, (project_root / 'Data' / 'Modelli_ML' / modello_per_analisi))

Valutazione delle performance sul test set
 - Mean Squared Error: 10268429.53
 - Root Mean Squared Error: 3204.44 euro
 - R-squared:  0.79


In [7]:
plot_is_conveniente_distribution(data_ml)


Distribuzione relativa della variabile 'is_conveniente':

 is_conveniente  Frequenza Relativa
              0            0.556387
              1            0.443613


In [8]:
plot_price_prediction_distributions(data_ml)

In [9]:
#is_conveniente_class_report(data_ml)

### 3. Indice di attendibilità

In [10]:
#Prezzo soglia: prezzo oltre il quale il punteggio di attendibilità diminuisce
prezzo_soglia=35000

#Valori per il calcolo indice di attendibilità
valore_idx_anni = 0.025
valore_idx_prezzo = 0.15
valore_idx_km= 0.15 
valore_idx_dist = 0.10 
valore_idx_allestimento= 0.25 
valore_idx_cv =0.025
valore_idx_cambio = 0.20 

In [11]:
df = indice_appetibilita(data, y_pred, prezzo_soglia, valore_idx_anni, valore_idx_prezzo, valore_idx_km, 
                           valore_idx_dist, valore_idx_allestimento, valore_idx_cv, valore_idx_cambio, 
                           allestimento_performance, allestimento_sport,allestimento_middle,allestimento_base)

In [12]:
prezzo_max = 32000
prezzo_min = 24000
km_max =40000
km_min = 0
dist_max = 150
dispaly = 20

display_top_auto(df, dispaly, prezzo_max, prezzo_min, km_max, km_min, dist_max, carburante=None)


Le migliori 20 auto secondo l'indice di appetibilità sono:


,Annuncio,Link,Prezzo,Anni,Immatricolazione,Chilometraggio,Cambio,Carburante,CV,Allestimento,...,Venditore,Distanza,Comune,CAP,Regione,Area,Indice_Appetibilità,prezzo_previsto,delta_prezzo,is_conveniente
233,BMW 116 i Msport,https://www.autoscout24.it/annunci/bmw-116-i-m...,27400,2,2023,31,automatico,Benzina,136,M Sport,...,Rivenditore,16,modena,41123,Emilia-Romagna,Nord-est,0.81,28411.91,1011.91,1
1937,Mercedes-Benz Classe A - D M SPORT AUT.,https://autosupermarket.it/annunci/bmw-serie-1...,29900,1,2024,14,automatico,Diesel,150,M Sport,...,Rivenditore,59,lendinara,45026,Veneto,Nord-est,0.81,31363.30,1463.30,1
1945,Mercedes-Benz Classe A - D M SPORT AUT.,https://autosupermarket.it/annunci/bmw-serie-1...,30900,1,2024,6,automatico,Diesel,150,M Sport,...,Rivenditore,59,lendinara,45026,Veneto,Nord-est,0.81,36663.73,5763.73,1
3,BMW 118 118i M Sport auto VARI COLORI DISPONIBILI,https://www.autoscout24.it/annunci/bmw-118-118...,26500,3,2022,17,automatico,Benzina,136,M Sport,...,Rivenditore,77,arcole,37040,Veneto,Nord-est,0.80,26652.00,152.00,1
5,BMW 118 I M-SPORT LED NAVI VARI COLORI DISPONI...,https://www.autoscout24.it/annunci/bmw-118-i-m...,26500,3,2022,17,automatico,Benzina,136,M Sport,...,Rivenditore,77,arcole,37040,Veneto,Nord-est,0.80,26652.00,152.00,1
21,"BMW 118 I Msport C.Lega17"" LED Clima2Zone M-Sp...",https://www.autoscout24.it/annunci/bmw-118-i-m...,26900,3,2022,22,automatico,Benzina,136,M Sport,...,Rivenditore,93,campi bisenzio,50013,Toscana,Centro,0.80,26993.80,93.80,0
29,BMW 118 118 i 5p. Msport M-sport *SCONTO IMMED...,https://www.autoscout24.it/annunci/bmw-118-118...,29890,1,2024,9,automatico,Benzina,136,M Sport,...,Rivenditore,101,padova,35129,Veneto,Nord-est,0.80,30945.55,1055.55,1
35,BMW 118 118 i 5p. Msport M-sport *SCONTO IMMED...,https://www.autoscout24.it/annunci/bmw-118-118...,29890,1,2024,8,automatico,Benzina,136,M Sport,...,Rivenditore,101,padova,35129,Veneto,Nord-est,0.80,30480.57,590.57,1
48,BMW 116 116d 5p. Msport,https://www.autoscout24.it/annunci/bmw-116-116...,26500,3,2022,59,automatico,Diesel,136,M Sport,...,Rivenditore,23,carpi,41012,Emilia-Romagna,Nord-est,0.80,27127.56,627.56,1
54,"BMW 118 bz Msport CAMBIO AUTO CERCHI 17,NAVI,L...",https://www.autoscout24.it/annunci/bmw-118-bz-...,29900,1,2024,21,automatico,Benzina,136,M Sport,...,Rivenditore,91,acquafredda,25010,Lombardia,Nord-ovest,0.80,29425.80,-474.20,0
